# SAFT-VR-Mie with polar contributions

In [ ]:
import teqp
teqp.__version__

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math, json

In [ ]:
# These values are not important, get something on the right order of magnitude
ek = 100 # [K]
sigma_m = 1e-10
                     
N_A = 6.022e23
fig, (ax1, ax2) = plt.subplots(2, 1)

kB = 1.380649e-23 # Boltzmann's constant, J/K
epsilon_0 = 8.8541878128e-12 # Vacuum permittivity
k_e = 1.0/(4.0*np.pi*epsilon_0*sigma_m**3)

polar_model = 'GrayGubbins+GubbinsTwu'

for mustar in [1, 2]:
    x,TT,DD = [],[],[]
    for alphastar in [0.0, 0.03, 0.06]:

        alpha_m3 = alphastar*sigma_m**3

        rhostar_guess = 0.27
        Tstar_guess = 1.5
        mu_Cm = (ek*kB/k_e)**0.5*mustar
        j = {
            "kind": 'SAFT-VR-Mie',
            "model": {
                "polar_model": polar_model,
                "polar_flags": {
                    "polarizable": {
                        "alpha_symm / m^3": [alpha_m3], 
                        "alpha_asymm / m^3": [0.0]
                    }
                },
                "coeffs": [{
                    "name": "PolarizableStockmayer",
                    "BibTeXKey": "me",
                    "m": 1.0,
                    "epsilon_over_k": ek, # [K]
                    "sigma_m": sigma_m,
                    "lambda_r": 12.0,
                    "lambda_a": 6.0,
                    "mu_Cm": mu_Cm,
                    "nmu": 1.0
                }]
            }
        }
        model = teqp.make_model(j)

        T, rho = model.solve_pure_critical(Tstar_guess*ek, rhostar_guess/(N_A*sigma_m**3))
        # Store the values
        x.append(alphastar)
        TT.append(T/ek)
        DD.append(rho*N_A*sigma_m**3)
        # Update the guess for the next calculation
        Tstar_guess = TT[-1]
        rhostar_guess = DD[-1]
#         print(TT[-1], DD[-1])

    ax1.plot(x, TT, label=f'$\mu^*={mustar}$')
    ax2.plot(x, DD)
        
ax1.legend(loc='best')
ax1.set(ylabel=r'$T^*$')
ax2.set(xlabel=r'$\alpha^*$', ylabel=r'$\rho^*$')
plt.show()